# Tutorial 1: Data preparation for Ghana adaptation example



In [13]:
#Load librariesimport os 

import sys 
import geopandas as gpd 
import pandas as pd 
%matplotlib notebook 
import matplotlib.pyplot as plt 
import subprocess 
import csv 
import json 
import math 
import os 
from collections import namedtuple, OrderedDict 
import fiona 
import geopandas as gpd 
import matplotlib.patches as mpatches 
import matplotlib.pyplot as plt 
import numpy as np 
from osgeo import gdal 
import shapely.geometry 
import shapely.ops 
from osgeo import gdal 
from scipy.spatial import Voronoi 
from shapely.geometry import Polygon, shape 

In [14]:
#Lena to input libraries

## Activity 1: Extract infrastructure data

#### Step 1) ...

In [19]:
#data folder link for MAC
data_folder = "/Users/lena/Desktop/Ghana_tutorial/"

#data folder link for Windows users
#data_folder = "XX"
#Sarah to input how it looks for Windows

#### Step 2) ...

#### Step 3) ...

#### Step 4) Load the road dataset you've just downloaded

In [21]:
GHA_OSM_roads = gpd.read_file(os.path.join(data_folder,'ghana-latest-free.shp', 'gis_osm_roads_free_1.shp'))

#### Step 5) To take a look at the data and the attribute table fill in and run the next two cells

In [24]:
GHA_OSM_roads.head()

,osm_id,code,fclass,name,ref,oneway,maxspeed,layer,bridge,tunnel,geometry
0,4790591,5121,unclassified,None,None,F,0,0,F,F,"LINESTRING (-0.17184 5.60847, -0.17182 5.60849..."
1,4790592,5122,residential,Nortei Ababio Road,None,B,0,0,F,F,"LINESTRING (-0.18282 5.61197, -0.18336 5.61198..."
2,4790594,5115,tertiary,None,None,F,0,0,F,F,"LINESTRING (-0.17544 5.60550, -0.17418 5.60555..."
3,4790596,5121,unclassified,None,None,F,0,0,F,F,"LINESTRING (-0.17207 5.60853, -0.17207 5.60844..."
4,4790597,5122,residential,Volta Road,None,B,0,0,F,F,"LINESTRING (-0.18282 5.61197, -0.18280 5.61262..."


#### Step 6) Next we want to make a couple of changes to the data

In [25]:
GHA_OSM_roads = GHA_OSM_roads.to_crs("EPSG:32630")

In [27]:
GHA_OSM_roads = GHA_OSM_roads[['fclass','name','oneway','bridge','tunnel','geometry']]
GHA_OSM_roads = GHA_OSM_roads.reset_index()
GHA_OSM_roads = GHA_OSM_roads.rename(columns = 
                                     {'index':'ID','fclass':'TYPE','name':'NAME'})

In [28]:
GHA_OSM_roads.head()

,ID,TYPE,NAME,oneway,bridge,tunnel,geometry
0,0,unclassified,None,F,F,F,"LINESTRING (813332.180 620680.284, 813334.396 ..."
1,1,residential,Nortei Ababio Road,B,F,F,"LINESTRING (812113.032 621062.316, 812053.497 ..."
2,2,tertiary,None,F,F,F,"LINESTRING (812935.075 620350.037, 813074.292 ..."
3,3,unclassified,None,F,F,F,"LINESTRING (813307.141 620687.468, 813307.057 ..."
4,4,residential,Volta Road,B,F,F,"LINESTRING (812113.032 621062.316, 812114.773 ..."


#### Step 7) Save the pre-processed dataset

In [29]:
GHA_OSM_roads.to_file(os.path.join(
data_folder,'GHA_OSM_roads.gpkg'),layer='OSM-roads', driver="GPKG")

## Activity 2: Extract and polygonise hazard data

In [ ]:
#As a second step, we will download the hazard flood layers, provided by Aquaduct in the form of tiff files. These need to be converted (poligonised) into shapefiles in order to perform the intersection between hazards and infrastructure assets. To extract and poligonise the flood hazards, follow these steps:  
#1) Download a few tif files from Aqueduct:  https://github.com/nismod/aqueduct/blob/main/tiffs.txt 

In [31]:
for root, dirs, files in os.walk(data_folder, 'flood_layer'): 
    print(root) 
    print(dirs) 

for file in files: 
    if file.endswith("tif"): 
        print("Found tif file",file[:-4]) 
        output_file = file[:-4] + ".shp" 
        print(output_file) 
        output_file_path = os.path.join(data_folder, output_file) 
        input_file_path = os.path.join(data_folder, file) 
        print(output_file_path) 
        subprocess.run(["gdal_polygonize.py", input_file_path,'-q','-f', 'ESRI SHAPEFILE',output_file_path]) 

/Users/lena/Desktop/Ghana_tutorial
['ghana-latest-free.shp', 'flood_layer']
/Users/lena/Desktop/Ghana_tutorial/ghana-latest-free.shp
[]
/Users/lena/Desktop/Ghana_tutorial/flood_layer
[]
Found tif file inuncoast_rcp4p5_nosub_2050_rp0050_0
inuncoast_rcp4p5_nosub_2050_rp0050_0.shp
/Users/lena/Desktop/Ghana_tutorial/inuncoast_rcp4p5_nosub_2050_rp0050_0.shp
Found tif file inuncoast_rcp8p5_wtsub_2080_rp0050_0
inuncoast_rcp8p5_wtsub_2080_rp0050_0.shp
/Users/lena/Desktop/Ghana_tutorial/inuncoast_rcp8p5_wtsub_2080_rp0050_0.shp
Found tif file inuncoast_rcp8p5_wtsub_2050_rp0500_0
inuncoast_rcp8p5_wtsub_2050_rp0500_0.shp
/Users/lena/Desktop/Ghana_tutorial/inuncoast_rcp8p5_wtsub_2050_rp0500_0.shp


## Activity 3: Intersect hazard ...

In [4]:
#### Step 1) Specify where you want to save your output file

In [32]:
output_path = gpd.read_file(os.path.join(data_folder, 'results','intersect' ) 

SyntaxError: unexpected EOF while parsing (<ipython-input-32-2aeb761916fe>, line 1)

In [ ]:
def line_length(line, ellipsoid='WGS-84'): 

    """Length of a line in meters, given in geographic coordinates 

    Adapted from https://gis.stackexchange.com/questions/4022/looking-for-a-pythonic-way-to-calculate-the-length-of-a-wkt-linestring#answer-115285 

    Args: 

        line: a shapely LineString object with WGS-84 coordinates. 

        ellipsoid: string name of an ellipsoid that `geopy` understands (see http://geopy.readthedocs.io/en/latest/#module-geopy.distance). 

    Returns: 

        Length of line in kilometers. 

    """ 

    if line.geometryType() == 'MultiLineString': 
        return sum(line_length(segment) for segment in line) 
    return sum( 
        vincenty(tuple(reversed(a)), tuple(reversed(b)), ellipsoid=ellipsoid).kilometers 
        for a, b in pairwise(line.coords) 

    ) 

def intersections(line_geodataframe,flood_geodataframe,output_path): 
    data=[] 
    poly_sindex = flood_gpd.sindex 
    for l,lines in road_gpd.iterrows(): 
        if lines.geometry: 
            intersected_polys = flood_gpd.iloc[list(poly_sindex.intersection(lines.geometry.bounds))] 
            for p,poly in intersected_polys.iterrows(): 
                if (lines['geometry'].intersects(poly['geometry']) is True) and (poly.geometry.is_valid is True): 
                    geom = lines['geometry'].intersection(poly['geometry']) 
                    data.append({'gid':lines['gid'],'flood_depth':poly['DN'],'flood_length':geom.length,'geometry':geom}) 

    if data: 

        intersections_data = gpd.GeoDataFrame(data, columns=['gid','flood_depth', 'flood_length', 'geometry'], crs='epsg:4326') 
        intersections_data.to_file(os.path.join(output_path,'flood_results.shp')) 
        return intersections_data 

def convert(threshold, infile, outfile): 

    """Convert GeoTiff raster file to Shapefile with geometries based on raster threshold less that 999 

    Parameters 

        - threshold - Float value of lower bound of GeoTiff threshold value to be selected 

        - infile - String name of input GeoTff file path 

        - tmpfile_1 - Stirng name of tmp file 1 

        - outfile - Stirng name of output shapefile 

    Outputs 

        Shapefile with Polygon geometries of rasters based on raster values above a threshold 

    """ 

    subprocess.run([ 
        "gdal_polygonize.py", 
        tmpfile_1, 
        '-q', 
        '-f', 'ESRI Shapefile', 
        outfile 
    ]) 

In [ ]:
intersections(GHA_OSM_roads,flood_geodataframe,output_path) 

## Activity 4: QGIS

Activity 4 is done in QGIS ...